In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
generative = combinators.ParticleTrace(1)

In [4]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [5]:
ssm_step = combinators.Model(ssm.ssm_step)
ssm_step.condition(trace=generative, guide=None)

In [6]:
for t in range(zs.shape[1] - 1):
    zs[:, t+1], mu, sigma, delta = ssm_step((zs[:, t], mu, sigma, delta), t)

In [7]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [8]:
num_particles = 100

In [9]:
smc_runner = smc.smc(ssm.ssm_step, 50)

In [10]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [11]:
init_ssm = combinators.Model(ssm.init_ssm, phi=ssm_params, theta={})

In [12]:
inference, init_ssm_params = smc.variational_smc(num_particles, init_ssm, smc_runner, 500, generative,
                                                 use_cuda=True, lr=1e-2)

07/29/2018 11:46:09 Variational SMC ELBO=-2.49829453e+04 at epoch 1
07/29/2018 11:46:09 Variational SMC ELBO=-1.81819473e+04 at epoch 2
07/29/2018 11:46:10 Variational SMC ELBO=-2.16102949e+04 at epoch 3
07/29/2018 11:46:11 Variational SMC ELBO=-1.71759941e+04 at epoch 4
07/29/2018 11:46:11 Variational SMC ELBO=-1.94582676e+04 at epoch 5
07/29/2018 11:46:12 Variational SMC ELBO=-1.92930625e+04 at epoch 6
07/29/2018 11:46:12 Variational SMC ELBO=-2.53136250e+04 at epoch 7
07/29/2018 11:46:13 Variational SMC ELBO=-2.06500859e+04 at epoch 8
07/29/2018 11:46:13 Variational SMC ELBO=-1.92247539e+04 at epoch 9
07/29/2018 11:46:13 Variational SMC ELBO=-2.04137656e+04 at epoch 10
07/29/2018 11:46:14 Variational SMC ELBO=-1.73622266e+04 at epoch 11
07/29/2018 11:46:14 Variational SMC ELBO=-2.35517539e+04 at epoch 12
07/29/2018 11:46:15 Variational SMC ELBO=-2.23687383e+04 at epoch 13
07/29/2018 11:46:16 Variational SMC ELBO=-2.24028672e+04 at epoch 14
07/29/2018 11:46:16 Variational SMC ELBO=-1

In [13]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.008926
SMC MSE at time 2: 0.930799
SMC MSE at time 3: 0.013823
SMC MSE at time 4: 1.255163
SMC MSE at time 5: 0.423680
SMC MSE at time 6: 9.685618
SMC MSE at time 7: 0.852883
SMC MSE at time 8: 0.188022
SMC MSE at time 9: 0.008569
SMC MSE at time 10: 0.006533
SMC MSE at time 11: 1.524233
SMC MSE at time 12: 0.447843
SMC MSE at time 13: 1.573699
SMC MSE at time 14: 0.207406
SMC MSE at time 15: 0.205839
SMC MSE at time 16: 1.121877
SMC MSE at time 17: 0.694331
SMC MSE at time 18: 0.215144
SMC MSE at time 19: 0.150537
SMC MSE at time 20: 0.035231
SMC MSE at time 21: 1.593911
SMC MSE at time 22: 0.578035
SMC MSE at time 23: 0.232760
SMC MSE at time 24: 2.385216
SMC MSE at time 25: 6.144553
SMC MSE at time 26: 0.000001
SMC MSE at time 27: 0.881045
SMC MSE at time 28: 0.640863
SMC MSE at time 29: 1.656077
SMC MSE at time 30: 4.668739
SMC MSE at time 31: 0.964409
SMC MSE at time 32: 0.410710
SMC MSE at time 33: 0.689576
SMC MSE at time 34: 5.303488
SMC MSE at time 35: 0.2

In [14]:
samples, elbos, inference = smc.particle_mh(num_particles, combinators.Model(ssm.init_ssm, theta=ssm_params), smc_runner, 500, generative)

In [15]:
inference

Trace{'mu': Normal([torch.FloatTensor of size 100]), 'sigma': Normal([torch.FloatTensor of size 100]), 'delta': Normal([torch.FloatTensor of size 100]), 'Z_0': Normal([torch.FloatTensor of size 100]), 'Z_1': Normal([torch.FloatTensor of size 100]), 'X_1': Normal([torch.FloatTensor of size 100x1x1]), 'Z_2': Normal([torch.FloatTensor of size 100]), 'X_2': Normal([torch.FloatTensor of size 100x1x1]), 'Z_3': Normal([torch.FloatTensor of size 100]), 'X_3': Normal([torch.FloatTensor of size 100x1x1]), 'Z_4': Normal([torch.FloatTensor of size 100]), 'X_4': Normal([torch.FloatTensor of size 100x1x1]), 'Z_5': Normal([torch.FloatTensor of size 100]), 'X_5': Normal([torch.FloatTensor of size 100x1x1]), 'Z_6': Normal([torch.FloatTensor of size 100]), 'X_6': Normal([torch.FloatTensor of size 100x1x1]), 'Z_7': Normal([torch.FloatTensor of size 100]), 'X_7': Normal([torch.FloatTensor of size 100x1x1]), 'Z_8': Normal([torch.FloatTensor of size 100]), 'X_8': Normal([torch.FloatTensor of size 100x1x1]),

In [16]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [17]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 3.255015
SMC MSE at time 2: 7.746152
SMC MSE at time 3: 0.131560
SMC MSE at time 4: 0.481298
SMC MSE at time 5: 0.007885
SMC MSE at time 6: 1.240737
SMC MSE at time 7: 4.231473
SMC MSE at time 8: 6.246520
SMC MSE at time 9: 0.076572
SMC MSE at time 10: 0.772442
SMC MSE at time 11: 0.134794
SMC MSE at time 12: 1.549202
SMC MSE at time 13: 10.110334
SMC MSE at time 14: 2.417177
SMC MSE at time 15: 0.599887
SMC MSE at time 16: 0.097397
SMC MSE at time 17: 0.255721
SMC MSE at time 18: 1.626658
SMC MSE at time 19: 0.001388
SMC MSE at time 20: 0.242282
SMC MSE at time 21: 4.286644
SMC MSE at time 22: 1.184104
SMC MSE at time 23: 0.008841
SMC MSE at time 24: 2.344586
SMC MSE at time 25: 0.627694
SMC MSE at time 26: 0.436644
SMC MSE at time 27: 1.527267
SMC MSE at time 28: 0.105531
SMC MSE at time 29: 1.862926
SMC MSE at time 30: 0.132800
SMC MSE at time 31: 0.755804
SMC MSE at time 32: 2.023236
SMC MSE at time 33: 0.331714
SMC MSE at time 34: 0.872143
SMC MSE at time 35: 0.

In [18]:
elbos

tensor([-15735.8213, -15735.8213, -15735.8213, -15735.8213, -15735.8213,
        -15735.8213, -15735.8213, -15735.8213, -15490.2715, -15490.2715,
        -15490.2715, -15490.2715, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -12870.8408, -12870.8408,
        -12870.8408, -12870.8408, -12870.8408, -128